In [1]:
import json as js


In [2]:
import os
import re

import pandas as pd

from process import *
from tokamak_helpers import *

In [3]:
def get_metadata(file):
    name_re = re.compile(r"pre-(\w+-)?(\w+)-(\d)-(\d+)")
    match = name_re.search(file)
    if match:
        return{"pre": match.group(2),
               "min_l": int(match.group(3)),
               "nodes": int(match.group(4)),
               "agg": bool(match.group(1))}
    else:
        raise RuntimeError

In [4]:
def process_log(filename):
    min_dofs = float("inf")
    max_dofs = 0
    iters = []
    with open(filename) as file:
        for line in file:
            dofs_re = re.compile(r"\)\s+(\d+)\s+\|\s+\d+\s+\|\s+(\d+)")
            it_re = re.compile(r"\[Ginkgo CG]\s+converged\s+after\s+(\d+)")
            if match := dofs_re.search(line):
                dof = int(match.group(2))
                max_dofs = max(max_dofs, dof)
                min_dofs = min(min_dofs, dof)
            elif match := it_re.search(line):
                iters.append(int(match.group(1)))
    return {**get_metadata(filename), "min_dofs": min_dofs, "max_dofs":max_dofs,
            "gmg_it": len(iters), "it": sum(iters)}

In [8]:

jsons = [f.path for f in os.scandir("gko-pre") if f.name.endswith(".json")]
logs = [f.path for f in os.scandir("gko-pre") if f.name.endswith(".log")]

In [9]:
db = Database(jsons, get_metadata)

In [10]:
it = pd.DataFrame([process_log(f) for f in logs])
it = it.set_index(["pre", "min_l", "nodes", "agg"])
it

min_dofs    max_dofs  gmg_it    it
pre        min_l nodes agg                                     
gko_ilu    1     2     True       766   114825259       7    70
           2     16    True      4818   912571536       9   201
                 8     True      2246   406155182      10   150
           3     128   True     32997  2978101016       9   402
                 64    True     15040  3234745378      10   320
gko_jacobi 1     2     True       766   114825259       7   172
           2     16    True      4818   912571536       9   501
                 8     True      2246   406155182      10   390
           3     128   True     32997  2978101016       9  1066
                 64    True     15040  3234745378      10   874

In [14]:
solve = db.get_df("solve").drop(columns=["node"])
solve = solve.groupby(["pre", "min_l", "nodes", "agg"]).median()
solve = pd.concat([solve, it], axis=1)
solve = solve.query("min_l < 4 and agg == True")
solve

average   count        max        min  \
pre        min_l nodes agg                                             
gko_ilu    1     2     True  23.880671   152.0  23.880682  23.880641   
           2     8     True  34.320119   608.0  34.320128  34.320075   
                 16    True  33.417559  1216.0  33.417595  33.417529   
           3     64    True  35.514466  4864.0  35.515920  35.514349   
                 128   True  39.213067  9728.0  39.215623  39.211280   
gko_jacobi 1     2     True  25.772929   152.0  25.772942  25.772885   
           2     8     True  32.923537   608.0  32.923553  32.923382   
                 16    True  33.564529  1216.0  33.564546  33.561706   
           3     64    True  36.279286  4864.0  36.282522  36.279240   
                 128   True  37.701596  9728.0  37.705285  37.698664   

                                     total      variance  min_dofs  \
pre        min_l nodes agg                                           
gko_ilu    1     2     True    3629.862032  4.087458e-11       766   
           2     8     True   20866.632118  1.614231e-11      2246   
                 16    True   40635.752342  1.355646e-11      4818   
           3     64    True  172742.362878  1.750578e-09     15040   
                 128   True  381464.716242  1.583629e-08     32997   
gko_jacobi 1     2     True    3917.485268  3.333843e-11       766   
           2     8     True   20017.510539  5.176753e-11      2246   
                 16    True   40814.467643  6.572234e-09      4818   
           3     64    True  176462.446429  2.200221e-09     15040   
                 128   True  366761.128757  2.353598e-09     32997   

                               max_dofs  gmg_it    it  
pre        min_l nodes agg                             
gko_ilu    1     2     True   114825259       7    70  
           2     8     True   406155182      10   150  
                 16    True   912571536       9   201  
           3     64    True  3234745378      10   320  
                 128   True  2978101016       9   402  
gko_jacobi 1     2     True   114825259       7   172  
           2     8     True   406155182      10   390  
                 16    True   912571536       9   501  
           3     64    True  3234745378      10   874  
                 128   True  2978101016       9  1066

In [15]:
solve.average / solve.gmg_it

pre         min_l  nodes  agg 
gko_ilu     1      2      True    3.411524
            2      8      True    3.432012
                   16     True    3.713062
            3      64     True    3.551447
                   128    True    4.357007
gko_jacobi  1      2      True    3.681847
            2      8      True    3.292354
                   16     True    3.729392
            3      64     True    3.627929
                   128    True    4.189066
dtype: float64

In [12]:
cgc = db.get_df("coarse grid").drop(columns=["node"])
cgc = cgc.groupby(["pre", "min_l", "nodes", "agg"]).median()
cgc = pd.concat([cgc, it], axis=1)
cgc.query("min_l < 4")["average"].unstack("agg")

agg                         True
pre        min_l nodes          
gko_ilu    1     2      0.012522
           2     8      0.020693
                 16     0.036351
           3     64     0.064822
                 128    0.127488
gko_jacobi 1     2      0.007626
           2     8      0.015490
                 16     0.020905
           3     64     0.043464
                 128    0.057044

In [13]:
pd.DataFrame(cgc.average / cgc.it).query("min_l < 4 and agg == True")

0
pre        min_l nodes agg           
gko_ilu    1     2     True  0.000179
           2     8     True  0.000138
                 16    True  0.000181
           3     64    True  0.000203
                 128   True  0.000317
gko_jacobi 1     2     True  0.000044
           2     8     True  0.000040
                 16    True  0.000042
           3     64    True  0.000050
                 128   True  0.000054